In [0]:
import pyspark.sql.functions as F

In [0]:
tables_to_count = [
    # "d_dnst_010_bronze.csv_export.c4s_business_partner_relationship_dump",
    # "d_dnst_010_bronze.csv_export.c4s_dump",

    # "d_dnst_010_bronze.sharepoint.agora_culinair_van_hout_5525957_v0",
    # "d_dnst_010_bronze.sharepoint.agora_culinair_vlb_5525741_v0",
    # "d_dnst_010_bronze.sharepoint.agora_culinair_vleminckx_5525965_v0",
    # "d_dnst_010_bronze.sharepoint.hanos_5525672_v0",
    # "d_dnst_010_bronze.sharepoint.hanos_5525672_v1",
    # "d_dnst_010_bronze.sharepoint.ilis_5526147_v0",
    # "d_dnst_010_bronze.sharepoint.makady_5525924_v0",
    # "d_dnst_010_bronze.sharepoint.solucious_5525843_v0",
    # "d_dnst_010_bronze.sharepoint.spuntini_aalter_5526188_v0",
    # "d_dnst_010_bronze.sharepoint.spuntini_biebuyck_5525668_v0",
    # "d_dnst_010_bronze.sharepoint.spuntini_deerlijk_5525976_v0",
    # "d_dnst_010_bronze.sharepoint.spuntini_deerlijk_5525976_v1",
    # "d_dnst_010_bronze.sharepoint.spuntini_degroof_5526040_v0",
    # "d_dnst_010_bronze.sharepoint.spuntini_free_foods_5525973_v0",
    # "d_dnst_010_bronze.sharepoint.spuntini_free_foods_5525973_v1",
    # "d_dnst_010_bronze.sharepoint.spuntini_ghislenghien_5525976_v0",
    # "d_dnst_010_bronze.sharepoint.van_zon_5526010_v0",

    # "d_dnst_020_silver.silver_01.`01_c4s_dump`",
    # "d_dnst_020_silver.silver_01.`01_sellout_consolidated`",

    "d_dnst_020_silver.silver_02.`02_c4s_dump`",
    "d_dnst_020_silver.silver_02.`02_sellout_consolidated`",
    "d_dnst_020_silver.silver_02.c4s_dumps",
    "d_dnst_020_silver.silver_02.dim_sellout_mapping_cust",
    "d_dnst_020_silver.silver_02.fact_sales",

    "d_dnst_020_silver.silver_02.`02_c4s_filtered_place_id`",
    "d_dnst_020_silver.silver_03.`02_c4s_with_place_id`",
    "d_dnst_020_silver.silver_03.`03_sellout_consolidated_place_id`",
    "d_dnst_020_silver.silver_03.`03_sellout_customers`",
    "d_dnst_020_silver.silver_03.`03_sellout_filtered_place_id`",
    "d_dnst_020_silver.silver_03.`03_sellout_sales`",
    "d_dnst_020_silver.silver_03.`03_sellout_with_place_id`",
    "d_dnst_020_silver.silver_03.`03_sellout_with_place_id`",
    "d_dnst_020_silver.silver_03.`031_sellout_customers_int_matched`",
    "d_dnst_020_silver.silver_03.`032_sellout_customers_ext_matched`",
    "d_dnst_020_silver.silver_03.google_api_mapping_table",
]

In [0]:
# List to hold the individual count DataFrames
count_dfs = []

for full_table_name in tables_to_count:
    # 1. Clean the table name for parsing
    clean_table_name = full_table_name.replace("`", "")
    
    # 2. Split the name into catalog, schema, and table name
    parts = clean_table_name.split('.')
    if len(parts) < 3:
        print(f"Skipping malformed table name: {full_table_name}")
        continue
        
    catalog, schema, table_name = parts[0], parts[1], ".".join(parts[2:])
    
    # Re-introduce backticks to the table name if it was originally there for complex names
    if full_table_name.count('`') > 0:
        table_name = table_name.strip('`')
    
    # 3. Use Spark SQL to count records in the table
    try:
        # Execute the count query
        count_query = f"SELECT COUNT(1) AS record_count FROM {full_table_name}"
        count_df = spark.sql(count_query)
        
        # 4. Add the required metadata columns (catalog, schema, AND table_name)
        count_df = count_df.withColumn("catalog", F.lit(catalog)) \
                           .withColumn("schema", F.lit(schema)) \
                           .withColumn("table_name", F.lit(table_name)) \
                           .select(
                               F.col("catalog"), 
                               F.col("schema"), 
                               F.col("table_name"),
                               F.col("record_count")
                           )
        
        count_dfs.append(count_df)
        
    except Exception as e:
        # Handle cases where a table might not exist or be accessible
        print(f"Error counting records for {full_table_name}: {e}")
        # Create an error DataFrame to ensure the row still appears
        error_data = [(catalog, schema, table_name, 0)]
        error_schema = "catalog STRING, schema STRING, table_name STRING, record_count LONG"
        error_df = spark.createDataFrame(error_data, error_schema)
        count_dfs.append(error_df)


# --- 5. Combine All DataFrames ---

schema_definition = "catalog STRING, schema STRING, table_name STRING, record_count LONG"

if not count_dfs:
    final_df = spark.createDataFrame([], schema_definition)
else:
    # Initialize the union with the first DataFrame
    final_df = count_dfs[0]
    
    # Union the rest
    for i in range(1, len(count_dfs)):
        final_df = final_df.unionByName(count_dfs[i])

# --- 6. Order the Final Result ---

final_df_ordered = final_df.orderBy(F.col("catalog"), F.col("schema"), F.col("table_name"))

In [0]:
(
    final_df_ordered
    .display()
)